In [45]:
import pandas as pd
import numpy as np
import collections
import copy

In [2]:
def read_line_to_dict(line):
    line = line[:-1]
    sep_line = [l.split(',') for l in line.split(';')]
#     sep_line = [list(map(int, l.split(','))) for l in line.split(';')]
    
    sep_line[0] = list(map(int, sep_line[0]))
    if sep_line[1] == ['']:
        sep_line[1] = [np.nan]
    else:
        sep_line[1] = list(map(int, sep_line[1]))
        
    dict_of_lines = {'viewed': sep_line[0], 'purchased': sep_line[1]}
    return dict_of_lines


In [3]:
def read_file_to_dict(name):
    file = open(name)
    list_of_dict =[]
    for line in file:
        list_of_dict.append(read_line_to_dict(line))
    return list_of_dict

In [4]:
def delete_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [219]:
def sort_sessions(arr, type_of_sort, mode, freq=None):
    '''
    type_of_sort = ['viewed', 'purchased']
    mode = ['train','test']
    
    '''
    if(mode == 'train'):
        freq = collections.Counter()
        for i in range(0, len(arr)):
            freq.update(arr[i][type_of_sort])
    
    arr_cleaned = [session for session in arr if session['purchased'] != [np.nan]]
    
    sorted_arr = copy.deepcopy(arr_cleaned)
    for num, session in enumerate(sorted_arr):
        sorted_arr[num]['viewed'] = sorted(delete_duplicates(session['viewed']),
                                               key=lambda x: freq[x],
                                               reverse=True)
    if(mode == 'train'):
        return sorted_arr, freq
    elif(mode == 'test'):
        return sorted_arr

In [196]:
def count_metric(session, power, metric):
    viewed_power_items = copy.deepcopy(session['viewed'][:power])
    len_of_viewed_power_items = len(viewed_power_items)
    purchased_items = copy.deepcopy(session['purchased'])
#     print(viewed_power_items) 
    for item in purchased_items:
        viewed_power_items = [x for x in viewed_power_items if x != item]
#     print(purchased_items)
#     print((len_of_viewed_power_items - len(viewed_power_items)) / power)
    if metric == 'precision':
        return (len_of_viewed_power_items - len(viewed_power_items)) / power
    elif metric == 'recall':
        return (len_of_viewed_power_items - len(viewed_power_items)) / len(purchased_items)
    else:
        print("ERROR: Invalid matric!!!")
    
def average_metric(data, power, metric):
    sum_of_metric = 0
    for session in data:
        sum_of_metric += count_metric(session, power, metric)
    return sum_of_metric / len(data)

In [213]:
train = read_file_to_dict('coursera_sessions_train.txt')
test = read_file_to_dict('coursera_sessions_test.txt')

In [224]:
#ANSWER 1
sorted_data, freq_1 = sort_sessions(train, type_of_sort='viewed', mode='train')

print(round(average_metric(sorted_data, 1, metric='recall'), 2))
print(round(average_metric(sorted_data, 1, metric='precision'), 2))
print(round(average_metric(sorted_data, 5, metric='recall'), 2))
print(round(average_metric(sorted_data, 5, metric='precision'), 2))

0.44
0.51
0.82
0.21


In [225]:
#ANSWER 2
sorted_data = sort_sessions(test, type_of_sort='viewed', mode='test', freq=freq_1)

print(round(average_metric(sorted_data, 1, metric='recall'), 2))
print(round(average_metric(sorted_data, 1, metric='precision'), 2))
print(round(average_metric(sorted_data, 5, metric='recall'), 2))
print(round(average_metric(sorted_data, 5, metric='precision'), 2))

0.42
0.48
0.8
0.2


In [228]:
#ANSWER 3
sorted_data, freq_2 = sort_sessions(train, type_of_sort='purchased', mode='train')

print(round(average_metric(sorted_data, 1, metric='recall'), 2))
print(round(average_metric(sorted_data, 1, metric='precision'), 2))
print(round(average_metric(sorted_data, 5, metric='recall'), 2))
print(round(average_metric(sorted_data, 5, metric='precision'), 2))

0.69
0.8
0.93
0.25


In [229]:
#ANSWER 4
sorted_data = sort_sessions(test, type_of_sort='viewed', mode='test', freq=freq_2)

print(round(average_metric(sorted_data, 1, metric='recall'), 2))
print(round(average_metric(sorted_data, 1, metric='precision'), 2))
print(round(average_metric(sorted_data, 5, metric='recall'), 2))
print(round(average_metric(sorted_data, 5, metric='precision'), 2))

0.46
0.53
0.82
0.21
